In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.optimizers import SGD
from PIL import Image
import tensorflow as tf

# TensorFlowが認識しているGPUデバイスをリストアップ
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("TensorFlow GPU devices detected:")
    for gpu in gpus:
        print(gpu)
else:
    print("No GPU device found.")


TensorFlow GPU devices detected:
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [20]:
classes = ['men', 'women']
nb_classes = len(classes)

train_data_dir ='./dataset/train'
validation_data_dir ='./dataset/val'

nb_train_samples = 2000
nb_validation_samples = 500
img_width = 224
img_height = 224
batch_size = 16

In [21]:
train_datagen = ImageDataGenerator(rescale=1.0 / 255, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [22]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size
)
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size
    )

Found 2646 images belonging to 2 classes.
Found 663 images belonging to 2 classes.


In [23]:
input_tensor = Input(shape=(img_width, img_height, 3))
ResNet50 = ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor)

In [24]:
top_model = Sequential()
top_model.add(Flatten(input_shape=ResNet50.output_shape[1:]))
top_model.add(Dense(nb_classes, activation='softmax'))

In [25]:
model = Model(inputs=ResNet50.input, outputs=top_model(ResNet50.output))

In [26]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(lr=1e-3, momentum=0.9),
    metrics=['accuracy']
)

In [27]:
history = model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

Epoch 1/30
125/125 [==============================] - 33s 236ms/step - loss: 10.5944 - accuracy: 0.5775 - val_loss: 33704.8828 - val_accuracy: 0.5665
Epoch 2/30
125/125 [==============================] - 27s 212ms/step - loss: 6.2601 - accuracy: 0.5615 - val_loss: 0.6851 - val_accuracy: 0.5887
Epoch 3/30
125/125 [==============================] - 28s 223ms/step - loss: 4.3925 - accuracy: 0.5688 - val_loss: 0.6819 - val_accuracy: 0.5786
Epoch 4/30
125/125 [==============================] - 30s 241ms/step - loss: 4.0265 - accuracy: 0.5835 - val_loss: 0.6836 - val_accuracy: 0.5706
Epoch 5/30
125/125 [==============================] - 29s 228ms/step - loss: 2.8431 - accuracy: 0.6090 - val_loss: 0.6769 - val_accuracy: 0.5867
Epoch 6/30
125/125 [==============================] - 26s 205ms/step - loss: 1.7869 - accuracy: 0.5915 - val_loss: 0.6790 - val_accuracy: 0.5645
Epoch 7/30
125/125 [==============================] - 27s 214ms/step - loss: 1.8960 - accuracy: 0.6322 - val_loss: 47.5694 - 

In [28]:
model.save('model.h5')